In [41]:
from youtube_transcript_api import YouTubeTranscriptApi,TranscriptsDisabled
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_groq import ChatGroq
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_community.embeddings import OpenAIEmbeddings,HuggingFaceBgeEmbeddings


In [13]:
videoid="2caQ4j9oohE"  # Replace with your YouTube video ID
import subprocess
subprocess.run([
    "yt-dlp",
    "--skip-download",
    "--write-auto-subs",
    "--sub-lang", "en",
    "--sub-format", "vtt",
    "-o", f"{videoid}.%(ext)s",
    f"https://www.youtube.com/watch?v={videoid}"
])

CompletedProcess(args=['yt-dlp', '--skip-download', '--write-auto-subs', '--sub-lang', 'en', '--sub-format', 'vtt', '-o', '2caQ4j9oohE.%(ext)s', 'https://www.youtube.com/watch?v=2caQ4j9oohE'], returncode=0)

In [24]:
videoid = "2caQ4j9oohE"  # change to your video ID or file name
raw_text = Path(f"{videoid}.en.vtt").read_text(encoding="utf-8")
import re 
from pathlib import Path
cleaned = re.sub(r"WEBVTT[\s\S]*?(\d{2}:\d{2}:\d{2}\.\d{3})", r"\1", raw_text)
cleaned=re.sub(r"\d{2}:\d{2}:\d{2}\.\d{3}.*\n", " ", cleaned)
cleaned = re.sub(r"<[^>]+>", "", cleaned)
cleaned = re.sub(r"\s+", " ", cleaned).strip()
# Save to cleaned file
Path(f"{videoid}_cleaned.txt").write_text(cleaned, encoding="utf-8")
print(f"✅ Cleaned transcript saved as {videoid}_cleaned.txt")
print(f"📝 Preview:\n{cleaned[:400]}...")


✅ Cleaned transcript saved as 2caQ4j9oohE_cleaned.txt
📝 Preview:
I< I realized have the career and the life I realized have the career and the life that< that looks like the dream, but are you that looks like the dream, but are you really< really happy, Emma? Are you really really happy, Emma? Are you really healthy?< healthy? And have to admit to myself healthy? And have to admit to myself that< that I wasn't was one of the scariest that I wasn't was one of th...


In [35]:
splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
text = Path(f"{videoid}_cleaned.txt").read_text(encoding="utf-8")
chunks=splitter.create_documents([text])
print(len(chunks))
print(chunks[2].page_content[:400])

407
almost of why now? Why today? Why here? &gt;&gt; I< &gt;&gt; I think I mentioned but I read your book &gt;&gt; I think I mentioned but I read your book um< um because my my dear friend Nuper told um because my my dear friend Nuper told me< me that I should and every now and again me that I should and every now and again I< I would see you come up on my feed. I I would see you come up on my feed. I


In [42]:
embeddings=HuggingFaceBgeEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store=FAISS.from_documents(chunks,embeddings)


c:\Users\Mayer\OneDrive\Desktop\youtube chatbot\ytenv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Mayer\OneDrive\Desktop\youtube chatbot\ytenv\lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mayer\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run 

In [45]:
vector_store.index_to_docstore_id

{0: '513d2e4e-96f3-4abf-a980-6967ca1d13b2',
 1: '88683efe-b633-471b-8323-62b4adaed8b8',
 2: '1e68b2ef-af06-47af-a464-b86a45e353c4',
 3: '59540e5f-5ac0-4923-808c-32c53c3c21fc',
 4: 'b735d3f7-1de9-4581-b1ea-17adc9d4f02a',
 5: '349854cb-9f51-4ae8-b0d5-ca531870f7ea',
 6: '93fddde4-baf5-4c49-ad79-af7dd621fb0a',
 7: '36d69cdd-b786-41c2-8238-f15bc9c64208',
 8: '9e9974dc-259c-417d-aed1-bbdbc76f015c',
 9: 'b6b7b293-716e-4243-a25b-78d72c6c4282',
 10: '24db5baa-b183-4156-abc3-3c5b0809fb23',
 11: '73ebf63b-a841-4003-9599-679c4257f6ba',
 12: '4523fcfc-0c57-47ad-8944-db2008f9452a',
 13: 'a5a57f9e-d4eb-4f68-beee-41c9158080e3',
 14: 'c64cd159-bbff-42ba-bb04-e08ce8ca6f0d',
 15: 'c24bf827-2345-46db-9e61-b1a6f544f3a8',
 16: 'ebccb27b-cecf-40e3-858f-ad10a7c6b206',
 17: 'fc135684-909e-4153-81d9-595a8e9fe3ed',
 18: '63b9fa65-2fb0-4692-acac-5ceeca01d79a',
 19: 'd024f933-e2e5-4cf2-8bfd-a32719d48827',
 20: '7689c2c0-7f5c-4efb-99d2-1a7a1b80b6e0',
 21: '2793c3a3-551d-4e97-9d2b-c127a0ec18ae',
 22: '5574b1e3-f86f-

In [49]:
vector_store.get_by_ids(['41d29d78-ddd8-461b-80ea-a367fe4e25ce'])

[Document(id='41d29d78-ddd8-461b-80ea-a367fe4e25ce', metadata={}, page_content="It's amazing. &gt;&gt; Um< &gt;&gt; Um that's amazing. Well, thank you for &gt;&gt; Um that's amazing. Well, thank you for Thank< Thank you so much. This has been such an Thank you so much. This has been such an amazing< amazing conversation. If you love this amazing conversation. If you love this episode,< episode, you'll love my interview with episode, you'll love my interview with Dr.< Dr. Gabbor Mate on understanding your Dr. Gabbor Mate on understanding your trauma< trauma and how to heal emotional wounds trauma and how to heal emotional wounds to< to start moving on from the past. to start moving on from the past. &gt;&gt; Everything< &gt;&gt; Everything in nature grows only where &gt;&gt; Everything in nature grows only where it's< it's vulnerable. So, a tree doesn't grow it's vulnerable. So, a tree doesn't grow where< where it's hard and thick, does it? It where it's hard and thick, does it? It goes

In [51]:
retriever=vector_store.as_retriever(search_type="similarity",search_kwargs={"k":4})


In [54]:
retriever.invoke("who is emma watson")

[Document(id='fbe2f7ca-4958-45d6-958c-e2747eed07a2', metadata={}, page_content="know. It's it's it's nuts.< nuts. So I Yeah, I think part of also nuts. So I Yeah, I think part of also not< not feeling like Emma Watson is just not feeling like Emma Watson is just like< like the whole like glam squad culture like the whole like glam squad culture of< of of it all is of of it all is it's< it's intense. it's intense. It's< It's yeah it's so fascinating because It's yeah it's so fascinating because there's< there's almost like this this learning there's almost like this this learning of< of becoming of becoming &gt;&gt; you< &gt;&gt; you know becoming Emma Watson becoming &gt;&gt; you know becoming Emma Watson becoming you< you know being all the roles you play you know being all the roles you play and< and then it almost feels like what and then it almost feels like what you're< you're saying is there was a moment you you're saying is there was a moment you wanted< wanted to step off and u

In [55]:
import os
from dotenv import load_dotenv
load_dotenv()
llm=ChatGroq(model="openai/gpt-oss-120b", api_key=os.getenv("GROQ_API_KEY"))
prompt=PromptTemplate(
    template="""you are a helpful assistant 
             answer  only from the provided transcript context , if the context is insufficient , just say no 
             {context}
             Question : {question}
            """,
            input_variables=["context","question"]
)


In [67]:
question="Is anything related to Harry potter being dixussed here? if yes then what are the things"
retrivecall=retriever.invoke(question)

In [ ]:
context=" ".join([doc.page_content for doc in retrivecall])
context


"I think I had developed< developed on Harry Potter, which was developed on Harry Potter, which was that< that we were the people I worked with that we were the people I worked with were< were going to be my family and that we were going to be my family and that we were< were going to be lifelong friends. I were going to be lifelong friends. I came< came to work looking for friendship came to work looking for friendship and< and that was a very painful experience and that was a very painful experience for< for me outside of Harry Potter and in for me outside of Harry Potter and in Hollywood. Hollywood. Hollywood. like like like bonebreakingly< bonebreakingly painful. Um because most bonebreakingly painful. Um because most people< people don't come to those environments people don't come to those environments looking< looking for friendships. They're looking looking for friendships. They're looking for< for this is my chance. This is my role. for this is my chance. This is my role. that

In [69]:
final_prompt=prompt.invoke({"context":context,"question":question})

In [70]:
ans=llm.invoke(final_prompt)
print(ans)

content='Yes. The transcript discusses Harry\u202fPotter in several ways:\n\n- The speaker says they “developed … on Harry\u202fPotter” and mentions “Harry\u202fPotter had a family feel.”\n- They talk about how the casting process looked for the characters Harry, Hermione, and Ron, describing a “country‑wide search” to find those roles.\n- The school was asked to submit students who wanted to audition for the Harry\u202fPotter project, and the speaker was one of about twelve students invited to audition.' additional_kwargs={'reasoning_content': 'We need to answer only from the provided transcript context. The user asks: "Is anything related to Harry potter being dixussed here? if yes then what are the things". We must check if the transcript mentions Harry Potter. Yes, multiple mentions: "developed on Harry Potter", "Harry Potter had a family feel", "auditions ... to find Harry, Hermione, and Ron", etc. So answer: yes, Harry Potter is discussed, specifically references to developing on

In [71]:
ans.content

'Yes. The transcript discusses Harry\u202fPotter in several ways:\n\n- The speaker says they “developed … on Harry\u202fPotter” and mentions “Harry\u202fPotter had a family feel.”\n- They talk about how the casting process looked for the characters Harry, Hermione, and Ron, describing a “country‑wide search” to find those roles.\n- The school was asked to submit students who wanted to audition for the Harry\u202fPotter project, and the speaker was one of about twelve students invited to audition.'

In [72]:
from langchain_core.runnables import RunnableParallel,RunnablePassthrough,RunnableLambda
from langchain_core.output_parsers import StrOutputParser

In [73]:
def format_doc(rey):
    context=" ".join([doc.page_content for doc in rey])
    return context
    

In [74]:
"""here do cheeze ho rhin ... pehle toh context generate hoga fir question toh context retreiver se aayega and uska aise hi nhi bhej skte toh usske liye format wala function lagega and usko chain m add karne k liye use function banana hoga so isliye wo kiya h naki fir question 
   toh pehli chain m retriver chlega .. uska output fornAT_DOC M BHEJA JAYEGA
   AND QUESTION KO PASSTHROUGH KRNA HAI SINCE question ko input m dalkr output m bhi question chahiye toh it would be runnablepassthrough
   
  """
parallel_chain=RunnableParallel({
    'context':retriever | RunnableLambda(format_doc),
    'question':RunnablePassthrough()}
)

In [75]:
parser=StrOutputParser()
main_chain=parallel_chain|prompt|llm|parser

In [76]:
main_chain.invoke(" tell qualities of emma watson in pointers")

'- **Intense** – Described as being part of an “intense” glam‑squad culture.  \n- **Continuously learning** – Emphasizes the need to “constantly be learning” and “unlearn” aspects of herself.  \n- **Versatile/Multifaceted** – Talks about “being all the roles you play” and the blending of Emma Watson, Hermione, and other characters.  \n- **Self‑reflective** – Reflects on the tension between her public persona and personal identity, noting she “cannot kill her off completely.”  \n- **Creative/Expressive** – Wrote a play that helped friends and family understand her life experience.  \n- **Aware of fame’s impact** – Recognizes the “ability to be a famous person” and how it shapes her life.'